Breezy Hofmeister

In [775]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set option to display all columns
pd.set_option('display.max_columns', None)

##Introduction

This project examines the role of second-home ownership in shaping housing pressure and economic conditions across Wisconsin counties. While my prior work has largely focused on urban and market-driven settings, recent summers spent camping in northern Wisconsin prompted a different line of inquiry. Many small towns experience dramatic seasonal population swings, raising questions about how much of the local housing stock is occupied by full-time residents versus seasonal or recreational owners.

Beyond documenting the prevalence of second homes, this project investigates how high levels of seasonal housing are associated with outcomes that directly affect year-round residents, including housing availability, affordability, and income pressure. Using county-level data from the American Community Survey (ACS), I construct a panel dataset spanning 2013–2023 and develop a predictive model to assess how growth in second-home share relates to changes in local housing conditions over time. The goal is to identify which counties may face increasing housing pressure in the future as seasonal demand continues to grow.

**Hypothesis:** A higher concentration of seasonal housing within a county is associated with home value growth that outpaces median household income growth, increasing housing affordability pressure for full-time residents.

**Data Sourced From:** [United States Census Bureau](https://data.census.gov/table?q=population&g=040XX00US55$0500000&y=2023) and the [Wisconsin Department of Natural Resources](https://data-wi-dnr.opendata.arcgis.com/datasets/wi-dnr::county-boundaries-24k/about)


##Preparing the Data

In [776]:
#Alright, this is going to start off our path to hopefully get all our data in one table.
#The first step is getting all the tables sourced from the ACS survey
#I have them in a zip file that I will now unzip from github

url = "https://raw.githubusercontent.com/bhofmeister0304/soccer-transfer-regret-analysis/main/data/raw/Archive.zip"

# Download the zip
!wget -O Archive.zip {url}

# Unzip into folder 'wisconsin'
!unzip -o Archive.zip -d wisconsin

# List contents to confirm
import os
os.listdir('wisconsin')


#Saving the dataframes
demo13 = pd.read_csv('wisconsin/Demographics 2013.csv')
demo18 = pd.read_csv('wisconsin/Demographics 2018.csv')
demo23 = pd.read_csv('wisconsin/Demographics 2023.csv')
income13 = pd.read_csv('wisconsin/Median Income 2013.csv')
income18 = pd.read_csv('wisconsin/Median Income 2018.csv')
income23 = pd.read_csv('wisconsin/Median Income 2023.csv')
Units13 = pd.read_csv('wisconsin/Total Units 2013.csv')
Units18 = pd.read_csv('wisconsin/Total Units 2018.csv')
Units23 = pd.read_csv('wisconsin/Total Units 2023.csv')
Vacancy13 = pd.read_csv('wisconsin/Vacancy 2013.csv')
Vacancy18 = pd.read_csv('wisconsin/Vacancy 2018.csv')
Vacancy23 = pd.read_csv('wisconsin/Vacancy 2023.csv')
value13 = pd.read_csv('wisconsin/Median Value 2013.csv')
value18 = pd.read_csv('wisconsin/Median Value 2018.csv')
value23 = pd.read_csv('wisconsin/Median Value 2023.csv')

--2026-01-09 23:01:06--  https://raw.githubusercontent.com/bhofmeister0304/soccer-transfer-regret-analysis/main/data/raw/Archive.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48311 (47K) [application/zip]
Saving to: ‘Archive.zip’

Archive.zip         100%[===================>]  47.18K  --.-KB/s    in 0.006s  

2026-01-09 23:01:06 (8.12 MB/s) - ‘Archive.zip’ saved [48311/48311]

Archive:  Archive.zip
  inflating: wisconsin/Demographics 2013.csv  
  inflating: wisconsin/__MACOSX/._Demographics 2013.csv  
  inflating: wisconsin/Demographics 2018.csv  
  inflating: wisconsin/__MACOSX/._Demographics 2018.csv  
  inflating: wisconsin/Demographics 2023.csv  
  inflating: wisconsin/__MACOSX/._Demographics 2023.csv  
  inflating: wisconsin/Median Income 2013.cs

We have some serious data cleaning to do. First, we need a function that drops the margin of error columns and cleans up the column names.

In [777]:

import re
import pandas as pd

def clean_acs_dataframe(df):
    #Drop margin of error column
    df = df.loc[:, ~df.columns.str.contains("Margin of Error", case=False)]

    #Drop Nan rows
    df = df.dropna(how="all")

    #Cleaning the column names
    new_cols = []
    for col in df.columns:
        if "County" in col:
            match = re.search(r"(.*County)", col)
            new_cols.append(match.group(1))
        else:
            new_cols.append(col)

    df.columns = new_cols

    return df


In [778]:
#Implementing the function
dfs = [
    demo13, demo18, demo23,
    income13, income18, income23,
    Units13, Units18, Units23,
    Vacancy13, Vacancy18, Vacancy23,
    value13, value18, value23
]

cleaned_dfs = [clean_acs_dataframe(df) for df in dfs]

(
    demo13, demo18, demo23,
    income13, income18, income23,
    Units13, Units18, Units23,
    Vacancy13, Vacancy18, Vacancy23,
    value13, value18, value23
) = cleaned_dfs


In [779]:
#Okay this one is slightly redundant, but some of the tables had a little bit more going on
#The main goal of this was to make sure each county only has one column
import re

def clean_acs_dataframe_keep_first(df):
    #drop margin of error columns
    df = df.loc[:, ~df.columns.str.contains("Margin of Error", case=False)]

    # Drop NaN Rows
    df = df.dropna(how="all")

    #Clean column names
    cleaned_cols = []
    for col in df.columns:
        if "County" in col:
            match = re.search(r"(.*County)", col)
            cleaned_cols.append(match.group(1))
        else:
            cleaned_cols.append(col)  # keeps Unnamed: 0

    df.columns = cleaned_cols

    #Only keeping the first column name
    df = df.loc[:, ~df.columns.duplicated()]

    return df


In [780]:
#Implementing the function
demo13 = clean_acs_dataframe_keep_first(demo13)
demo18 = clean_acs_dataframe_keep_first(demo18)
demo23 = clean_acs_dataframe_keep_first(demo23)

income13 = clean_acs_dataframe_keep_first(income13)
income18 = clean_acs_dataframe_keep_first(income18)
income23 = clean_acs_dataframe_keep_first(income23)


In [781]:
# I am testing out this to see if this will swap the rows to columns and clean up the names
demo13 = demo13.T.reset_index()
demo13 = demo13.rename(columns = {'index':'County', 0:'total_population'})
demo13 = demo13.drop(demo13.index[0])
demo13['year'] = 2013

In [782]:
demo13.head()

,County,total_population,year
1,Adams County,"20,741",2013
2,Ashland County,"16,068",2013
3,Barron County,"45,826",2013
4,Bayfield County,"15,071",2013
5,Brown County,"250,597",2013


In [783]:
#And it does!
#So now I will be implementing it across all rows
demo18 = demo18.T.reset_index()
demo18 = demo18.rename(columns = {'index':'County', 0:'total_population'})
demo18 = demo18.drop(demo18.index[0])
demo18['year'] = 2018

demo23 = demo23.T.reset_index()
demo23 = demo23.rename(columns = {'index':'County', 0:'total_population'})
demo23 = demo23.drop(demo23.index[0])
demo23['year'] = 2023

income13 = income13.T.reset_index()
income13 = income13.rename(columns = {'index':'County', 0:'median_household_income'})
income13 = income13.drop(income13.index[0])
income13['year'] = 2013

income18 = income18.T.reset_index()
income18 = income18.rename(columns = {'index':'County', 0:'median_household_income'})
income18 = income18.drop(income18.index[0])
income18['year'] = 2018

income23 = income23.T.reset_index()
income23 = income23.rename(columns = {'index':'County', 0:'median_household_income'})
income23 = income23.drop(income23.index[0])
income23['year'] = 2023

Units13 = Units13.T.reset_index()
Units13 = Units13.rename(columns = {'index':'County', 0:'total_housing_units'})
Units13 = Units13.drop(Units13.index[0])
Units13['year'] = 2013

Units18 = Units18.T.reset_index()
Units18 = Units18.rename(columns = {'index':'County', 0:'total_housing_units'})
Units18 = Units18.drop(Units18.index[0])
Units18['year'] = 2018

Units23 = Units23.T.reset_index()
Units23 = Units23.rename(columns = {'index':'County', 0:'total_housing_units'})
Units23 = Units23.drop(Units23.index[0])
Units23['year'] = 2023

value13 = value13.T.reset_index()
value13 = value13.rename(columns = {'index':'County', 0:'median_home_value'})
value13 = value13.drop(value13.index[0])
value13['year'] = 2013

value18 = value18.T.reset_index()
value18 = value18.rename(columns = {'index':'County', 0:'median_home_value'})
value18 = value18.drop(value18.index[0])
value18['year'] = 2018

value23 = value23.T.reset_index()
value23 = value23.rename(columns = {'index':'County', 0:'median_home_value'})
value23 = value23.drop(value23.index[0])
value23['year'] = 2023





In [784]:
#Vacancy had a little extra cleaning to do
Vacancy13 = Vacancy13.drop(Vacancy13.index[0:5])
Vacancy13 = Vacancy13.drop(Vacancy13.index[1:3])

Vacancy18 = Vacancy18.drop(Vacancy18.index[0:5])
Vacancy18 = Vacancy18.drop(Vacancy18.index[1:3])

Vacancy23 = Vacancy23.drop(Vacancy23.index[0:5])
Vacancy23 = Vacancy23.drop(Vacancy23.index[1:3])
Vacancy23

,Label (Grouping),Adams County,Ashland County,Barron County,Bayfield County,Brown County,Buffalo County,Burnett County,Calumet County,Chippewa County,Clark County,Columbia County,Crawford County,Dane County,Dodge County,Door County,Douglas County,Dunn County,Eau Claire County,Florence County,Fond du Lac County,Forest County,Grant County,Green County,Green Lake County,Iowa County,Iron County,Jackson County,Jefferson County,Juneau County,Kenosha County,Kewaunee County,La Crosse County,Lafayette County,Langlade County,Lincoln County,Manitowoc County,Marathon County,Marinette County,Marquette County,Menominee County,Milwaukee County,Monroe County,Oconto County,Oneida County,Outagamie County,Ozaukee County,Pepin County,Pierce County,Polk County,Portage County,Price County,Racine County,Richland County,Rock County,Rusk County,St. Croix County,Sauk County,Sawyer County,Shawano County,Sheboygan County,Taylor County,Trempealeau County,Vernon County,Vilas County,Walworth County,Washburn County,Washington County,Waukesha County,Waupaca County,Waushara County,Winnebago County,Wood County
5,"For seasonal, recreational, or occasional use","6,473","1,755","3,104","5,198",862,360,"7,844",413,"1,327","1,087","1,523","1,366","1,826","1,096","8,720","2,671",345,619,"2,290",669,"4,292",749,187,"1,875",371,"2,172","1,055",995,"2,980","2,130",515,598,116,"2,647","3,096",890,787,"9,319","2,413",701,"1,354",670,"6,293","12,478",536,497,224,263,"4,583",605,"3,586","1,299",788,"1,206","1,949",608,"2,184","6,871","2,213","1,025","1,041",317,"1,102","12,727","8,453","4,725",974,"2,517","1,979","3,634",917,627


In [785]:
#Finishing up that
Vacancy13 = Vacancy13.T.reset_index()
Vacancy13 = Vacancy13.rename(columns = {'index':'County', 5:'total_seasonal_units'})
Vacancy13 = Vacancy13.drop(Vacancy13.index[0])
Vacancy13['year'] = 2013

Vacancy18 = Vacancy18.T.reset_index()
Vacancy18 = Vacancy18.rename(columns = {'index':'County', 5:'total_seasonal_units'})
Vacancy18 = Vacancy18.drop(Vacancy18.index[0])
Vacancy18['year'] = 2018

Vacancy23 = Vacancy23.T.reset_index()
Vacancy23 = Vacancy23.rename(columns = {'index':'County', 5:'total_seasonal_units'})
Vacancy23 = Vacancy23.drop(Vacancy23.index[0])
Vacancy23['year'] = 2023

Vacancy23

,County,total_seasonal_units,year
1,Adams County,"6,473",2023
2,Ashland County,"1,755",2023
3,Barron County,"3,104",2023
4,Bayfield County,"5,198",2023
5,Brown County,862,2023
...,...,...,...
68,Waukesha County,"2,517",2023
69,Waupaca County,"1,979",2023
70,Waushara County,"3,634",2023
71,Winnebago County,917,2023


In [786]:
#making one dataframe for each dataframe group
demo = pd.concat([demo13, demo18, demo23], ignore_index=True)
income = pd.concat([income13, income18, income23], ignore_index=True)
units = pd.concat([Units13, Units18, Units23], ignore_index=True)
vacancy = pd.concat([Vacancy13, Vacancy18, Vacancy23], ignore_index=True)
value = pd.concat([value13, value18, value23], ignore_index=True)



In [787]:
#And finally merging them all together
final_df = demo \
    .merge(income, on=['County', 'year'], how='inner') \
    .merge(units, on=['County', 'year'], how='inner') \
    .merge(vacancy, on=['County', 'year'], how='inner') \
    .merge(value, on=['County', 'year'], how='inner')


In [788]:
#Looking good!
final_df

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value
0,Adams County,"20,741",2013,"44,897","17,358","7,465","135,600"
1,Ashland County,"16,068",2013,"38,550","9,625","2,313","107,400"
2,Barron County,"45,826",2013,"44,054","23,598","2,952","134,900"
3,Bayfield County,"15,071",2013,"44,944","13,037","5,273","158,600"
4,Brown County,"250,597",2013,"53,119","105,151",531,"158,700"
...,...,...,...,...,...,...,...
211,Waukesha County,"409,040",2023,"104,100","173,567","2,517","373,600"
212,Waupaca County,"51,714",2023,"71,189","25,620","1,979","192,800"
213,Waushara County,"24,752",2023,"64,968","14,804","3,634","186,000"
214,Winnebago County,"171,357",2023,"72,873","76,770",917,"208,600"


In [789]:
#Making sure that the columns I need to be numeric are numeric
for col in final_df.columns:
    final_df[col] = (
        final_df[col]
        .astype(str)
        .str.replace(',', '', regex=False)
        .replace('nan', np.nan)
    )
    final_df[col] = pd.to_numeric(final_df[col], errors='ignore')


/tmp/ipython-input-2828009651.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  final_df[col] = pd.to_numeric(final_df[col], errors='ignore')
/tmp/ipython-input-2828009651.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  final_df[col] = pd.to_numeric(final_df[col], errors='ignore')
/tmp/ipython-input-2828009651.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  final_df[col] = pd.to_numeric(final_df[col], errors='ignore')
/tmp/ipython-input-2828009651.py:9: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  final_df[col] = pd

In [790]:
final_df

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value
0,Adams County,20741,2013,44897,17358,7465,135600
1,Ashland County,16068,2013,38550,9625,2313,107400
2,Barron County,45826,2013,44054,23598,2952,134900
3,Bayfield County,15071,2013,44944,13037,5273,158600
4,Brown County,250597,2013,53119,105151,531,158700
...,...,...,...,...,...,...,...
211,Waukesha County,409040,2023,104100,173567,2517,373600
212,Waupaca County,51714,2023,71189,25620,1979,192800
213,Waushara County,24752,2023,64968,14804,3634,186000
214,Winnebago County,171357,2023,72873,76770,917,208600


In [791]:
#Adding an important feature of the pct of seasonal units in the county
#This will be crucial for my analysis
final_df['pct_seasonal'] = round(
    (final_df['total_seasonal_units'] / final_df['total_housing_units']) * 100,
    2
)

final_df



,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal
0,Adams County,20741,2013,44897,17358,7465,135600,43.01
1,Ashland County,16068,2013,38550,9625,2313,107400,24.03
2,Barron County,45826,2013,44054,23598,2952,134900,12.51
3,Bayfield County,15071,2013,44944,13037,5273,158600,40.45
4,Brown County,250597,2013,53119,105151,531,158700,0.50
...,...,...,...,...,...,...,...,...
211,Waukesha County,409040,2023,104100,173567,2517,373600,1.45
212,Waupaca County,51714,2023,71189,25620,1979,192800,7.72
213,Waushara County,24752,2023,64968,14804,3634,186000,24.55
214,Winnebago County,171357,2023,72873,76770,917,208600,1.19


In [792]:
#Removing 'County' from each County name to make some more data joining easier
final_df['County'] = final_df['County'].str.replace(' County', '', regex=False)


In [793]:
final_df

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal
0,Adams,20741,2013,44897,17358,7465,135600,43.01
1,Ashland,16068,2013,38550,9625,2313,107400,24.03
2,Barron,45826,2013,44054,23598,2952,134900,12.51
3,Bayfield,15071,2013,44944,13037,5273,158600,40.45
4,Brown,250597,2013,53119,105151,531,158700,0.50
...,...,...,...,...,...,...,...,...
211,Waukesha,409040,2023,104100,173567,2517,373600,1.45
212,Waupaca,51714,2023,71189,25620,1979,192800,7.72
213,Waushara,24752,2023,64968,14804,3634,186000,24.55
214,Winnebago,171357,2023,72873,76770,917,208600,1.19


In [794]:
#I will need the FIP codes for future mapping, so now I am loading them in
url = "https://github.com/bhofmeister0304/soccer-transfer-regret-analysis/raw/refs/heads/main/data/raw/County_Boundaries_24K.csv"

county_fip = pd.read_csv(url)

county_fip

,OBJECTID,DNR_REGION_NAME,DNR_CNTY_CODE,COUNTY_NAME,COUNTY_FIPS_CODE,SHAPEAREA,SHAPELEN
0,1281,Southeast Region,30,Kenosha,59,7.210454e+08,123267.303358
1,1282,South Central Region,33,Lafayette,65,1.641795e+09,164707.649840
2,1283,South Central Region,54,Rock,105,1.879382e+09,174114.587746
3,1284,Southeast Region,65,Walworth,127,1.492598e+09,154833.279362
4,1285,South Central Region,23,Green,45,1.512855e+09,155741.104371
...,...,...,...,...,...,...,...
67,1348,Northern Region,26,Iron,51,2.076556e+09,241955.543402
68,1349,Northern Region,16,Douglas,31,3.476446e+09,258177.017534
69,1350,Northern Region,4,Bayfield,7,3.914443e+09,342451.980379
70,1351,Northern Region,2,Ashland,3,2.729952e+09,526588.065473


In [795]:
#Changing names of useful columns
county_fip = county_fip.rename(columns = {'COUNTY_NAME':'County', 'COUNTY_FIPS_CODE':'FIPS'})
county_fip

,OBJECTID,DNR_REGION_NAME,DNR_CNTY_CODE,County,FIPS,SHAPEAREA,SHAPELEN
0,1281,Southeast Region,30,Kenosha,59,7.210454e+08,123267.303358
1,1282,South Central Region,33,Lafayette,65,1.641795e+09,164707.649840
2,1283,South Central Region,54,Rock,105,1.879382e+09,174114.587746
3,1284,Southeast Region,65,Walworth,127,1.492598e+09,154833.279362
4,1285,South Central Region,23,Green,45,1.512855e+09,155741.104371
...,...,...,...,...,...,...,...
67,1348,Northern Region,26,Iron,51,2.076556e+09,241955.543402
68,1349,Northern Region,16,Douglas,31,3.476446e+09,258177.017534
69,1350,Northern Region,4,Bayfield,7,3.914443e+09,342451.980379
70,1351,Northern Region,2,Ashland,3,2.729952e+09,526588.065473


In [796]:
#Merge final_df and county_fip
final_df = final_df.merge(county_fip, on='County', how='inner')

final_df = final_df.drop(columns=['OBJECTID', 'DNR_REGION_NAME', 'DNR_CNTY_CODE', 'SHAPEAREA', 'SHAPELEN'])

#Now we have a nice looking dataframe!
final_df

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal,FIPS
0,Adams,20741,2013,44897,17358,7465,135600,43.01,1
1,Ashland,16068,2013,38550,9625,2313,107400,24.03,3
2,Barron,45826,2013,44054,23598,2952,134900,12.51,5
3,Bayfield,15071,2013,44944,13037,5273,158600,40.45,7
4,Brown,250597,2013,53119,105151,531,158700,0.50,9
...,...,...,...,...,...,...,...,...,...
211,Waukesha,409040,2023,104100,173567,2517,373600,1.45,133
212,Waupaca,51714,2023,71189,25620,1979,192800,7.72,135
213,Waushara,24752,2023,64968,14804,3634,186000,24.55,137
214,Winnebago,171357,2023,72873,76770,917,208600,1.19,139


## Data Exploration

In [797]:
#This is one of the most crucial steps in the whole project so far
#Getting summary statistics by year for our variables!
final_df.groupby('year').describe()

total_population                                                          \
                count          mean            std     min       25%      50%   
year                                                                            
2013             72.0  79262.097222  132967.547940  4292.0  19639.00  41251.0   
2018             72.0  80255.472222  135440.207515  4337.0  19062.25  40773.0   
2023             72.0  81833.652778  135308.954693  4256.0  19396.00  41417.0   

                         median_household_income                              \
           75%       max                   count          mean           std   
year                                                                           
2013  85361.00  950527.0                    72.0  49474.583333   7997.868098   
2018  87811.25  954209.0                    72.0  56147.486111   8972.707066   
2023  89699.75  927656.0                    72.0  71715.486111  10327.572041   

                                                     total_housing_units  \
          min       25%      50%       75%       max               count   
year                                                                       
2013  33333.0  44689.00  48709.5  53180.00   75850.0                72.0   
2018  40801.0  50904.50  55771.5  59564.50   84331.0                72.0   
2023  57258.0  64135.75  70946.5  76190.75  104100.0                72.0   

                                                                       \
              mean           std     min       25%      50%       75%   
year                                                                    
2013  36474.194444  56960.559058  2324.0  11994.25  21162.5  36508.25   
2018  37239.333333  57813.209619  2390.0  12132.00  21461.0  37269.00   
2023  38204.861111  60118.234176  2148.0  11903.25  22177.5  38211.25   

               total_seasonal_units                                           \
           max                count         mean          std    min     25%   
year                                                                           
2013  417415.0                 72.0  2490.250000  2825.706143  134.0  635.50   
2018  418375.0                 72.0  2660.611111  3015.179305  178.0  700.00   
2023  423838.0                 72.0  2440.013889  2791.834959  116.0  669.75   

                               median_home_value                               \
         50%      75%      max             count           mean           std   
year                                                                            
2013  1295.0  2913.00  12847.0              72.0  148761.111111  32061.558552   
2018  1409.5  3207.75  13838.0              72.0  157683.333333  34290.519365   
2023  1313.0  2748.25  12727.0              72.0  217831.944444  53313.013570   

                                                       pct_seasonal  \
           min       25%       50%       75%       max        count   
year                                                                  
2013   88500.0  128350.0  143950.0  159450.0  253800.0         72.0   
2018  103300.0  136100.0  152800.0  167025.0  273000.0         72.0   
2023   97000.0  184075.0  209350.0  242400.0  373600.0         72.0   

                                                                FIPS  \
           mean        std   min    25%    50%      75%    max count   
year                                                                   
2013  14.665278  16.215802  0.27  1.745  7.455  23.6400  56.77  72.0   
2018  14.997500  16.110770  0.40  1.890  8.280  23.4750  50.83  72.0   
2023  13.831111  15.272797  0.32  1.785  6.620  21.8825  51.35  72.0   

                                                           
           mean        std  min   25%   50%    75%    max  
year                                                       
2013  71.097222  40.996104  1.0  36.5  72.0  105.5  141.0  
2018  71.097222  40.996104  1.0  36.5  72.0  105.5  141.0  
2023  71.0972

### Initial Analysis – Key Findings

- Average county population increased slightly from 2013 to 2023, though population levels vary widely across counties, reflecting large differences between urban and rural areas.

- Median household income rose steadily across all three years, with the largest increase occurring between 2018 and 2023.

- Total housing units increased on average in each county over time, indicating continued expansion of the housing stock statewide.

- Total seasonal housing units increased from 2013 to 2018 but declined slightly from 2018 to 2023 on average.

- Median home values increased substantially across the state over the study period, with particularly strong growth between 2018 and 2023.

- The average percentage of housing units classified as seasonal rose slightly from 2013 to 2018 and then declined from 2018 to 2023, suggesting recent housing growth may be driven more by non-seasonal units in some counti




In [798]:
#Okay now lets see which counties are dominated by seasonal homes
top_seasonal = (
    final_df
    .sort_values(by=['year', 'pct_seasonal'], ascending=[True, False])
    .groupby('year')
    .head(5)
    [['year', 'County', 'pct_seasonal']]
)

top_seasonal

,year,County,pct_seasonal
18,2013,Florence,56.77
63,2013,Vilas,51.09
20,2013,Forest,50.92
57,2013,Sawyer,46.90
6,2013,Burnett,46.80
90,2018,Florence,50.83
135,2018,Vilas,50.71
78,2018,Burnett,49.62
92,2018,Forest,48.90
129,2018,Sawyer,48.45


## Findings

- The same five counties (Florence, Vilas, Forest, Burnett, and Sawyer) consistently rank in the top five for percentage of seasonal housing across all three years.
- All five counties are located in Northern Wisconsin, aligning with expectations given the region’s concentration of recreational land, lakes, and tourism activity.
- While these counties remain the most seasonal, each shows a lower percentage of seasonal housing in 2023 compared to 2013, indicating a modest decline in seasonality over time.

I think that the next step will be to look at a similar table but for total seasonal housing volume.

In [799]:
#Okay now lets see which counties are dominated by seasonal homes
top_seasonal_units = (
    final_df
    .sort_values(by=['year', 'total_seasonal_units'], ascending=[True, False])
    .groupby('year')
    .head(5)
    [['year', 'County', 'total_seasonal_units']]
)

top_seasonal_units

,year,County,total_seasonal_units
63,2013,Vilas,12847
43,2013,Oneida,12362
37,2013,Marinette,9939
14,2013,Door,8468
64,2013,Walworth,8119
115,2018,Oneida,13838
135,2018,Vilas,13028
109,2018,Marinette,10040
136,2018,Walworth,9193
86,2018,Door,9030


###Findings
- Now we can see that the top five counties are a little different than the mere percentages
- However, we can also see that the top 5 counties in total seasonal units are also unchanged over the course of the 10 years

In [800]:
#Lets go back to the big picture, now we can see the numbers for the state as a whole and the percentage
state_units_per_year = (
    final_df
    .groupby('year', as_index=False)
    .agg(
        total_housing_units_state=('total_housing_units', 'sum'),
        total_seasonal_units_state=('total_seasonal_units', 'sum')
    )
)

state_units_per_year['seasonal_pct'] = round(
    (state_units_per_year['total_seasonal_units_state'] / state_units_per_year['total_housing_units_state']) * 100,
    2
)

state_units_per_year

,year,total_housing_units_state,total_seasonal_units_state,seasonal_pct
0,2013,2626142,179298,6.83
1,2018,2681232,191564,7.14
2,2023,2750750,175681,6.39


In [801]:
growth_df = (
    final_df
    .pivot(index='County', columns='year', values=[
        'total_seasonal_units',
        'total_housing_units',
        'median_household_income',
        'median_home_value',
        'pct_seasonal'
    ])
)

# Flatten column names
growth_df.columns = ['_'.join(map(str, col)) for col in growth_df.columns]
growth_df = growth_df.reset_index()

# Growth calculations
growth_df['income_growth_pct'] = (
    (growth_df['median_household_income_2023'] -
     growth_df['median_household_income_2013']) /
     growth_df['median_household_income_2013']
) * 100

growth_df['housing_units_growth_pct'] = (
    (growth_df['total_housing_units_2023'] -
     growth_df['total_housing_units_2013']) /
     growth_df['total_housing_units_2013']
) * 100

growth_df['seasonal_units_growth_pct'] = (
    (growth_df['total_seasonal_units_2023'] -
     growth_df['total_seasonal_units_2013']) /
     growth_df['total_seasonal_units_2013']
) * 100

growth_df['home_value_growth_pct'] = (
    (growth_df['median_home_value_2023'] -
     growth_df['median_home_value_2013']) /
     growth_df['median_home_value_2013']
) * 100

growth_df['growth_gap'] = (
    growth_df['home_value_growth_pct'] -
    growth_df['income_growth_pct']
)

growth_df.head()

#sorting growth_df by growth gap ascending
growth_df = growth_df.sort_values(by='growth_gap', ascending=True)
growth_df.head(30)

#In this case we would want a growth gap of near 0, that would mean that home prices in the county are growing steadily with wages
#Here I highlight the counties where that is not the case, and it turns out my home county of La Crosse is having the hardest time
#Interestingly, some of the biggest seasonal counties in the state are the ones who are seeing incomes grow faster than home values


,County,total_seasonal_units_2013,total_seasonal_units_2018,total_seasonal_units_2023,total_housing_units_2013,total_housing_units_2018,total_housing_units_2023,median_household_income_2013,median_household_income_2018,median_household_income_2023,median_home_value_2013,median_home_value_2018,median_home_value_2023,pct_seasonal_2013,pct_seasonal_2018,pct_seasonal_2023,income_growth_pct,housing_units_growth_pct,seasonal_units_growth_pct,home_value_growth_pct,growth_gap
39,Menominee,1045.0,873.0,701.0,2324.0,2390.0,2148.0,33333.0,43872.0,59528.0,88500.0,103300.0,97000.0,44.97,36.53,32.64,78.585786,-7.573150,-32.918660,9.604520,-68.981266
63,Vilas,12847.0,13028.0,12727.0,25145.0,25691.0,24783.0,40833.0,44285.0,67132.0,186000.0,206900.0,270700.0,51.09,50.71,51.35,64.406240,-1.439650,-0.934070,45.537634,-18.868606
69,Waushara,4001.0,4154.0,3634.0,14846.0,15060.0,14804.0,43070.0,51195.0,64968.0,136800.0,143100.0,186000.0,26.95,27.58,24.55,50.842814,-0.282904,-9.172707,35.964912,-14.877902
43,Oneida,12362.0,13838.0,12478.0,30142.0,30914.0,30746.0,45759.0,54198.0,69621.0,165200.0,167800.0,227500.0,41.01,44.76,40.58,52.147119,2.003848,0.938359,37.711864,-14.435254
15,Douglas,2697.0,3339.0,2671.0,22772.0,23124.0,22997.0,45418.0,52480.0,72579.0,130600.0,143700.0,190400.0,11.84,14.44,11.61,59.802281,0.988056,-0.964034,45.788668,-14.013613
37,Marinette,9939.0,10040.0,9319.0,30296.0,30719.0,29372.0,40490.0,47497.0,63401.0,108200.0,117100.0,154600.0,32.81,32.68,31.73,56.584342,-3.049908,-6.238052,42.883549,-13.700793
3,Bayfield,5273.0,5679.0,5198.0,13037.0,13306.0,13285.0,44944.0,52910.0,69609.0,158600.0,166300.0,228100.0,40.45,42.68,39.13,54.879405,1.902278,-1.422340,43.820933,-11.058472
46,Pepin,269.0,306.0,224.0,3583.0,3669.0,3603.0,47701.0,53140.0,74536.0,134800.0,148700.0,196100.0,7.51,8.34,6.22,56.256682,0.558191,-16.728625,45.474777,-10.781905
56,Sawyer,7478.0,7916.0,6871.0,15945.0,16338.0,16114.0,39904.0,44555.0,59055.0,162500.0,164700.0,223300.0,46.90,48.45,42.64,47.992682,1.059893,-8.117144,37.415385,-10.577298
20,Forest,4578.0,4474.0,4292.0,8990.0,9149.0,8696.0,39963.0,44313.0,59727.0,122400.0,130200.0,170000.0,50.92,48.90,49.36,49.455747,-3.270300,-6.247270,38.888889,-10.566858


## Key Conclusions

- Counties with high shares of seasonal housing generally experienced slower home value growth relative to income growth. Many northern Wisconsin counties (such as Menominee, Vilas, Oneida, and Waushara) show negative growth gaps, indicating that median household income increased faster than median home values between 2013 and 2023.

- Menominee County stands out as a major outlier. It experienced nearly 79% growth in median household income but only about 10% growth in median home values, resulting in a large negative growth gap. This suggests that housing markets in some high-seasonality or structurally constrained counties may not respond strongly to broader housing price trends.

- High-seasonality counties appear partially insulated from extreme housing price escalation. Counties with seasonal housing shares above 40–50% often saw home value growth lag income growth, indicating that seasonal demand does not uniformly translate into rapid price inflation, particularly in more remote or lower-growth areas.

- In contrast, low-seasonality counties tend to experience faster home value growth than income growth. Urban and suburban counties such as Dane, Ozaukee, St. Croix, and Calumet have very low seasonal housing shares and positive growth gaps, pointing to increasing housing affordability pressure on full-time residents.

- Housing pressure across Wisconsin appears to be driven more by economic growth, population dynamics, and proximity to metropolitan areas than by seasonal housing alone. Permanent demand, rather than seasonal demand, plays a larger role in driving long-term housing price appreciation.

- Overall, a high concentration of seasonal housing does not automatically imply worsening affordability for residents. The impact of seasonal homes on housing markets varies significantly by region and broader economic conditions.

**Hypothesis Implications:** The data is looking like it might actually prove my hypothesis wrong! Counties with high seasonality appear to have incomes consistently outpacing home value growth which should not price out full-time residents. Let's run a simple regression to see if this is true.

In [802]:
import statsmodels.formula.api as smf

#Formula
formula = "growth_gap ~ pct_seasonal_2023"
model = smf.ols(formula=formula, data=growth_df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             growth_gap   R-squared:                       0.261
Model:                            OLS   Adj. R-squared:                  0.250
Method:                 Least Squares   F-statistic:                     24.67
Date:                Fri, 09 Jan 2026   Prob (F-statistic):           4.63e-06
Time:                        23:01:08   Log-Likelihood:                -272.76
No. Observations:                  72   AIC:                             549.5
Df Residuals:                      70   BIC:                             554.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             5.9732      1.72

### Regression Results

An OLS regression was estimated to test whether the share of seasonal housing is associated with the gap between home price growth and income growth across Wisconsin counties.

The results indicate a statistically significant negative relationship between seasonal housing share and the growth gap. A one percentage-point increase in the share of seasonal housing is associated with a 0.42 percentage-point decrease in the home price–income growth gap (p < 0.001).

Because the growth gap is defined as home value growth minus income growth, this result implies that in counties with higher levels of seasonal housing, incomes tend to grow faster relative to home values. This finding runs counter to the initial hypothesis and suggests that seasonal housing may not exacerbate long-term affordability pressures for full-time residents in the way originally expected.

**Next Steps:** I suppose that sometimes it is okay to be wrong! It looks like there may be some investigating to do. I am going to add another feature that will be a ratio of median household income and median home price. This is a common affordability index, and this should allow us to see if things are actually improving in these seasonal counties.


In [803]:
growth_df['affordability_index_2013'] = (growth_df['median_home_value_2013'] / growth_df['median_household_income_2013'])
growth_df['affordability_index_2018'] = (growth_df['median_home_value_2018'] / growth_df['median_household_income_2018'])
growth_df['affordability_index_2023'] = (growth_df['median_home_value_2023'] / growth_df['median_household_income_2023'])
growth_df

growth_df['affordability_index_growth_pct'] = (
    (growth_df['affordability_index_2023'] -
     growth_df['affordability_index_2013']) /
     growth_df['affordability_index_2013']
) * 100

filtered_df_seasonal = growth_df[growth_df['pct_seasonal_2023'] > 10]
filtered_df_regular = growth_df[growth_df['pct_seasonal_2023'] < 10]

In [804]:
filtered_df_regular

,County,total_seasonal_units_2013,total_seasonal_units_2018,total_seasonal_units_2023,total_housing_units_2013,total_housing_units_2018,total_housing_units_2023,median_household_income_2013,median_household_income_2018,median_household_income_2023,median_home_value_2013,median_home_value_2018,median_home_value_2023,pct_seasonal_2013,pct_seasonal_2018,pct_seasonal_2023,income_growth_pct,housing_units_growth_pct,seasonal_units_growth_pct,home_value_growth_pct,growth_gap,affordability_index_2013,affordability_index_2018,affordability_index_2023,affordability_index_growth_pct
46,Pepin,269.0,306.0,224.0,3583.0,3669.0,3603.0,47701.0,53140.0,74536.0,134800.0,148700.0,196100.0,7.51,8.34,6.22,56.256682,0.558191,-16.728625,45.474777,-10.781905,2.825937,2.798269,2.630943,-6.900124
30,Kewaunee,563.0,557.0,515.0,9294.0,9414.0,9325.0,53588.0,63118.0,80085.0,149000.0,157700.0,208900.0,6.06,5.92,5.52,49.445771,0.333549,-8.525755,40.201342,-9.244429,2.780473,2.498495,2.608478,-6.185808
27,Jefferson,1195.0,1265.0,995.0,35134.0,35694.0,36649.0,53454.0,63676.0,80604.0,176900.0,184300.0,253800.0,3.40,3.54,2.71,50.791335,4.312062,-16.736402,43.470888,-7.320447,3.309388,2.894340,3.148727,-4.854687
40,Milwaukee,1128.0,1679.0,1354.0,417415.0,418375.0,423838.0,43193.0,48742.0,62118.0,158400.0,153600.0,216500.0,0.27,0.40,0.32,43.814970,1.538756,20.035461,36.679293,-7.135677,3.667261,3.151286,3.485302,-4.961707
9,Clark,1047.0,1282.0,1087.0,15035.0,15151.0,14842.0,43276.0,51872.0,66250.0,111700.0,121300.0,163800.0,6.96,8.46,7.32,53.087161,-1.283671,3.820439,46.642793,-6.444368,2.581107,2.338448,2.472453,-4.209607
55,Sauk,2540.0,2491.0,2184.0,29676.0,30295.0,31091.0,52140.0,56732.0,77648.0,168800.0,174500.0,242400.0,8.56,8.22,7.02,48.922133,4.768163,-14.015748,43.601896,-5.320237,3.237438,3.075865,3.121780,-3.572496
5,Buffalo,495.0,529.0,360.0,6689.0,6736.0,6546.0,47384.0,57134.0,68722.0,140900.0,155900.0,197600.0,7.40,7.85,5.50,45.032078,-2.137838,-27.272727,40.241306,-4.790772,2.973578,2.728673,2.875353,-3.303250
66,Washington,782.0,500.0,974.0,54878.0,56328.0,58766.0,66159.0,74062.0,95851.0,222900.0,225800.0,316200.0,1.42,0.89,1.66,44.879759,7.084806,24.552430,41.857335,-3.022424,3.369156,3.048797,3.298870,-2.086160
35,Manitowoc,622.0,925.0,890.0,37114.0,37428.0,37960.0,48881.0,53489.0,68611.0,125200.0,127200.0,172900.0,1.68,2.47,2.34,40.363331,2.279463,43.086817,38.099042,-2.264290,2.561322,2.378059,2.520004,-1.613163
16,Dunn,522.0,476.0,345.0,17937.0,18399.0,18812.0,48893.0,56268.0,71785.0,156200.0,161800.0,226200.0,2.91,2.59,1.83,46.820608,4.878185,-33.908046,44.814341,-2.006268,3.194731,2.875524,3.151076,-1.366476


In [805]:
filtered_df_seasonal

,County,total_seasonal_units_2013,total_seasonal_units_2018,total_seasonal_units_2023,total_housing_units_2013,total_housing_units_2018,total_housing_units_2023,median_household_income_2013,median_household_income_2018,median_household_income_2023,median_home_value_2013,median_home_value_2018,median_home_value_2023,pct_seasonal_2013,pct_seasonal_2018,pct_seasonal_2023,income_growth_pct,housing_units_growth_pct,seasonal_units_growth_pct,home_value_growth_pct,growth_gap,affordability_index_2013,affordability_index_2018,affordability_index_2023,affordability_index_growth_pct
39,Menominee,1045.0,873.0,701.0,2324.0,2390.0,2148.0,33333.0,43872.0,59528.0,88500.0,103300.0,97000.0,44.97,36.53,32.64,78.585786,-7.573150,-32.918660,9.604520,-68.981266,2.655027,2.354577,1.629485,-38.626403
63,Vilas,12847.0,13028.0,12727.0,25145.0,25691.0,24783.0,40833.0,44285.0,67132.0,186000.0,206900.0,270700.0,51.09,50.71,51.35,64.406240,-1.439650,-0.934070,45.537634,-18.868606,4.555139,4.672011,4.032354,-11.476818
69,Waushara,4001.0,4154.0,3634.0,14846.0,15060.0,14804.0,43070.0,51195.0,64968.0,136800.0,143100.0,186000.0,26.95,27.58,24.55,50.842814,-0.282904,-9.172707,35.964912,-14.877902,3.176225,2.795195,2.862948,-9.863182
43,Oneida,12362.0,13838.0,12478.0,30142.0,30914.0,30746.0,45759.0,54198.0,69621.0,165200.0,167800.0,227500.0,41.01,44.76,40.58,52.147119,2.003848,0.938359,37.711864,-14.435254,3.610219,3.096055,3.267692,-9.487695
15,Douglas,2697.0,3339.0,2671.0,22772.0,23124.0,22997.0,45418.0,52480.0,72579.0,130600.0,143700.0,190400.0,11.84,14.44,11.61,59.802281,0.988056,-0.964034,45.788668,-14.013613,2.875512,2.738186,2.623348,-8.769345
37,Marinette,9939.0,10040.0,9319.0,30296.0,30719.0,29372.0,40490.0,47497.0,63401.0,108200.0,117100.0,154600.0,32.81,32.68,31.73,56.584342,-3.049908,-6.238052,42.883549,-13.700793,2.672265,2.465419,2.438447,-8.749785
3,Bayfield,5273.0,5679.0,5198.0,13037.0,13306.0,13285.0,44944.0,52910.0,69609.0,158600.0,166300.0,228100.0,40.45,42.68,39.13,54.879405,1.902278,-1.422340,43.820933,-11.058472,3.528836,3.143073,3.276875,-7.140053
56,Sawyer,7478.0,7916.0,6871.0,15945.0,16338.0,16114.0,39904.0,44555.0,59055.0,162500.0,164700.0,223300.0,46.90,48.45,42.64,47.992682,1.059893,-8.117144,37.415385,-10.577298,4.072273,3.696555,3.781221,-7.147176
20,Forest,4578.0,4474.0,4292.0,8990.0,9149.0,8696.0,39963.0,44313.0,59727.0,122400.0,130200.0,170000.0,50.92,48.90,49.36,49.455747,-3.270300,-6.247270,38.888889,-10.566858,3.062833,2.938190,2.846284,-7.070225
6,Burnett,7102.0,7684.0,7844.0,15176.0,15485.0,15328.0,39564.0,49982.0,61664.0,147600.0,156100.0,215500.0,46.80,49.62,51.17,55.858862,1.001581,10.447761,46.002710,-9.856152,3.730664,3.123124,3.494746,-6.323767


In [806]:
filtered_df_seasonal.describe()

,total_seasonal_units_2013,total_seasonal_units_2018,total_seasonal_units_2023,total_housing_units_2013,total_housing_units_2018,total_housing_units_2023,median_household_income_2013,median_household_income_2018,median_household_income_2023,median_home_value_2013,median_home_value_2018,median_home_value_2023,pct_seasonal_2013,pct_seasonal_2018,pct_seasonal_2023,income_growth_pct,housing_units_growth_pct,seasonal_units_growth_pct,home_value_growth_pct,growth_gap,affordability_index_2013,affordability_index_2018,affordability_index_2023,affordability_index_growth_pct
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,4608.419355,4917.258065,4468.516129,16445.322581,16742.354839,16414.580645,43974.741935,50012.225806,64832.096774,136167.741935,143245.161290,193422.580645,29.778387,30.383226,28.195806,47.921312,-1.070683,-4.887963,41.360620,-6.560692,3.093006,2.864306,2.968559,-4.024467
std,3209.176309,3427.570052,3245.318483,9768.289186,9934.125868,10082.678423,4376.967146,5060.881173,5667.105530,25867.165636,26865.576968,42712.587603,14.073327,13.217567,13.050824,9.784373,2.942987,10.040729,8.659612,13.894522,0.487352,0.470142,0.517870,8.205340
min,976.000000,873.000000,701.000000,2324.000000,2390.000000,2148.000000,33333.000000,40801.000000,57258.000000,88500.000000,103300.000000,97000.000000,9.730000,10.590000,10.830000,30.913261,-10.262009,-32.918660,9.604520,-68.981266,2.547831,2.354577,1.629485,-38.626403
25%,2489.000000,2491.000000,2192.500000,9777.500000,9934.500000,9573.000000,40661.500000,45556.000000,59627.500000,118550.000000,124600.000000,167600.000000,17.895000,19.585000,18.070000,42.359011,-2.334184,-8.948422,37.678550,-10.572078,2.767992,2.518457,2.625866,-7.106828
50%,3319.000000,3540.000000,3104.000000,14664.000000,15060.000000,14639.000000,44149.000000,49982.000000,63496.000000,131100.000000,139800.000000,186500.000000,26.950000,28.300000,24.550000,47.992682,-0.745633,-1.422428,42.413793,-5.404116,2.875512,2.738186,2.853087,-3.441961
75%,6706.000000,7295.000000,6383.000000,23161.000000,23562.500000,23481.500000,46318.000000,53122.000000,67429.000000,147650.000000,155700.000000,213750.000000,42.010000,43.110000,38.670000,51.690061,0.994818,1.551801,45.895689,-0.084615,3.380168,3.063576,3.218034,-0.059359
max,12847.000000,13838.000000,12727.000000,51496.000000,52177.000000,53508.000000,54020.000000,61106.000000,77359.000000,194900.000000,207100.000000,305800.000000,56.770000,50.830000,51.350000,78.585786,3.907100,10.447761,59.105099,16.781851,4.555139,4.672011,4.259943,11.791363


In [807]:
filtered_df_regular.describe()

,total_seasonal_units_2013,total_seasonal_units_2018,total_seasonal_units_2023,total_housing_units_2013,total_housing_units_2018,total_housing_units_2023,median_household_income_2013,median_household_income_2018,median_household_income_2023,median_home_value_2013,median_home_value_2018,median_home_value_2023,pct_seasonal_2013,pct_seasonal_2018,pct_seasonal_2023,income_growth_pct,housing_units_growth_pct,seasonal_units_growth_pct,home_value_growth_pct,growth_gap,affordability_index_2013,affordability_index_2018,affordability_index_2023,affordability_index_growth_pct
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.00000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,888.707317,954.365854,906.268293,51617.975610,52737.048780,54680.439024,53633.000000,60786.341463,76920.000000,158282.926829,168600.000000,236287.804878,3.238293,3.364390,2.97000,43.706743,4.512197,6.114145,48.992358,5.285615,2.936132,2.759415,3.047333,3.796485
std,585.126492,628.636252,579.411168,71704.038567,72711.108068,75450.314481,7616.031982,8512.688828,10037.318143,33270.496708,35525.849744,53533.878036,2.709473,2.741272,2.33682,5.907795,4.057235,31.369495,7.726816,8.457278,0.299382,0.271629,0.356407,5.939034
min,134.000000,178.000000,116.000000,3583.000000,3669.000000,3603.000000,43193.000000,48742.000000,62118.000000,111700.000000,121300.000000,163800.000000,0.270000,0.400000,0.32000,28.197516,-3.730418,-33.908046,36.679293,-10.781905,2.480298,2.257291,2.472453,-6.900124
25%,432.000000,500.000000,515.000000,16160.000000,16495.000000,17052.000000,48893.000000,55832.000000,71189.000000,136300.000000,149500.000000,203300.000000,1.420000,1.500000,1.45000,39.282793,2.279463,-16.728625,43.470888,-0.522814,2.709030,2.575245,2.867922,-0.341908
50%,706.000000,801.000000,787.000000,34062.000000,34868.000000,34662.000000,52140.000000,58870.000000,74009.000000,152300.000000,159100.000000,226200.000000,2.090000,2.430000,1.86000,43.814970,4.121984,-3.094607,47.419355,4.891518,2.883631,2.738833,2.975819,3.422148
75%,1195.000000,1265.000000,1102.000000,54878.000000,56328.000000,58766.000000,55584.000000,63118.000000,80604.000000,168800.000000,173200.000000,247800.000000,4.400000,5.150000,3.38000,48.922133,6.535228,23.940150,54.579439,11.850398,3.119152,2.879542,3.177975,7.875043
max,2540.000000,2552.000000,2517.000000,417415.000000,418375.000000,423838.000000,75850.000000,84331.000000,104100.000000,253800.000000,273000.000000,373600.000000,9.820000,9.280000,9.23000,56.256682,17.399956,117.454545,69.018850,19.487636,3.707004,3.576643,4.155128,15.011625


###Affordability Findings

This actually provided some really interesting results. According to the [Wisconsin Policy Forum](https://wispolicyforum.org/research/home-prices-outpace-incomes/#:~:text=This%20divergence%20was%20particular%20acute,household%20income%20on%20a%20home.) home buyers can afford spending 2.6 times their annual household income on a home. So, an affordability index of 3 would be the upper end of what I would say is affordable in each county.

What I did was split up the counties of the state into two dataframes. One for seasonal counties (`pct_seasonal_2023`) > 10% and one for non-seasonal counties where it is < 10%.

Interestingly, just by looking at this and printing it out it appeared that the seasonal counties were becoming more affordable with almost all counties seeing an improvement in affordability from 2013 to 2023. Whereas in the non-seasonal counties it appeared to be the opposite with those counties becoming less affordable.

For the seasonal counties we saw a mean decline in the affordability ratio of ~4% whereas in the non-seasonal counties we saw an increase, on average, of ~3.79%.

This is interesting and I am not quite sure how to explain it, so for now I am going to do a little more research!

In [808]:
#Adding an indicator column if a county is a seasonal county if it has >10% of homes in 2023 as seasonal properties
growth_df['seasonal_indicator'] = growth_df['pct_seasonal_2023'].apply(lambda x: 1 if x > 10 else 0)
growth_df

,County,total_seasonal_units_2013,total_seasonal_units_2018,total_seasonal_units_2023,total_housing_units_2013,total_housing_units_2018,total_housing_units_2023,median_household_income_2013,median_household_income_2018,median_household_income_2023,median_home_value_2013,median_home_value_2018,median_home_value_2023,pct_seasonal_2013,pct_seasonal_2018,pct_seasonal_2023,income_growth_pct,housing_units_growth_pct,seasonal_units_growth_pct,home_value_growth_pct,growth_gap,affordability_index_2013,affordability_index_2018,affordability_index_2023,affordability_index_growth_pct,seasonal_indicator
39,Menominee,1045.0,873.0,701.0,2324.0,2390.0,2148.0,33333.0,43872.0,59528.0,88500.0,103300.0,97000.0,44.97,36.53,32.64,78.585786,-7.573150,-32.918660,9.604520,-68.981266,2.655027,2.354577,1.629485,-38.626403,1
63,Vilas,12847.0,13028.0,12727.0,25145.0,25691.0,24783.0,40833.0,44285.0,67132.0,186000.0,206900.0,270700.0,51.09,50.71,51.35,64.406240,-1.439650,-0.934070,45.537634,-18.868606,4.555139,4.672011,4.032354,-11.476818,1
69,Waushara,4001.0,4154.0,3634.0,14846.0,15060.0,14804.0,43070.0,51195.0,64968.0,136800.0,143100.0,186000.0,26.95,27.58,24.55,50.842814,-0.282904,-9.172707,35.964912,-14.877902,3.176225,2.795195,2.862948,-9.863182,1
43,Oneida,12362.0,13838.0,12478.0,30142.0,30914.0,30746.0,45759.0,54198.0,69621.0,165200.0,167800.0,227500.0,41.01,44.76,40.58,52.147119,2.003848,0.938359,37.711864,-14.435254,3.610219,3.096055,3.267692,-9.487695,1
15,Douglas,2697.0,3339.0,2671.0,22772.0,23124.0,22997.0,45418.0,52480.0,72579.0,130600.0,143700.0,190400.0,11.84,14.44,11.61,59.802281,0.988056,-0.964034,45.788668,-14.013613,2.875512,2.738186,2.623348,-8.769345,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12,Dane,1302.0,1689.0,1826.0,217104.0,229498.0,254880.0,61721.0,70541.0,88108.0,228800.0,252300.0,366100.0,0.60,0.74,0.72,42.752062,17.399956,40.245776,60.008741,17.256680,3.707004,3.576643,4.155128,12.088568,0
7,Calumet,319.0,319.0,413.0,19744.0,20485.0,22163.0,65130.0,72530.0,87700.0,164300.0,173200.0,252100.0,1.62,1.56,1.86,34.653769,12.251823,29.467085,53.438831,18.785062,2.522647,2.387977,2.874572,13.950640,0
45,Ozaukee,401.0,513.0,497.0,36341.0,37216.0,39415.0,75457.0,82807.0,96734.0,250200.0,273000.0,368900.0,1.10,1.38,1.26,28.197516,8.458766,23.940150,47.442046,19.244530,3.315796,3.296823,3.813551,15.011625,0
59,St. Croix,383.0,480.0,608.0,34062.0,35471.0,38185.0,68426.0,81124.0,102482.0,206900.0,233600.0,349700.0,1.12,1.35,1.59,49.770555,12.104398,58.746736,69.018850,19.248295,3.023704,2.879542,3.412307,12.851855,0


In [809]:
correlation_matrix = growth_df.drop(columns=['County']).corr()
print(correlation_matrix)

                                total_seasonal_units_2013  \
total_seasonal_units_2013                        1.000000   
total_seasonal_units_2018                        0.996447   
total_seasonal_units_2023                        0.994822   
total_housing_units_2013                        -0.111679   
total_housing_units_2018                        -0.112780   
total_housing_units_2023                        -0.119636   
median_household_income_2013                    -0.351490   
median_household_income_2018                    -0.366444   
median_household_income_2023                    -0.301288   
median_home_value_2013                           0.054635   
median_home_value_2018                           0.018077   
median_home_value_2023                          -0.015591   
pct_seasonal_2013                                0.741522   
pct_seasonal_2018                                0.777254   
pct_seasonal_2023                                0.781871   
income_growth_pct       

In [810]:
import statsmodels.formula.api as smf

# Define the formula
formula = """
affordability_index_growth_pct ~ pct_seasonal_2023 + seasonal_indicator +
income_growth_pct + home_value_growth_pct + housing_units_growth_pct + affordability_index_2013
"""

# Fit the model
model = smf.ols(formula=formula, data=growth_df).fit()

# Print results
print(model.summary())


                                  OLS Regression Results                                  
Dep. Variable:     affordability_index_growth_pct   R-squared:                       0.990
Model:                                        OLS   Adj. R-squared:                  0.989
Method:                             Least Squares   F-statistic:                     1086.
Date:                            Fri, 09 Jan 2026   Prob (F-statistic):           3.79e-63
Time:                                    23:01:08   Log-Likelihood:                -84.881
No. Observations:                              72   AIC:                             183.8
Df Residuals:                                  65   BIC:                             199.7
Df Model:                                       6                                         
Covariance Type:                        nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.97

### Regression Results

An OLS regression was estimated to examine whether seasonal housing and other county-level factors are associated with changes in housing affordability across Wisconsin counties from 2013 to 2023. Affordability is measured as the percent change in the affordability index, defined as **median home price ÷ median household income**.  

**Regression Coefficients:**

| Variable                     | Coefficient | Std. Error | t-value | p-value | Interpretation |
|-------------------------------|------------|-----------|--------|--------|----------------|
| Intercept                     | 1.182      | 1.055     | 1.120  | 0.267  | Baseline change in affordability |
| pct_seasonal_2023             | 0.002      | 0.013     | 0.163  | 0.871  | No significant effect of seasonal housing share |
| seasonal_indicator            | -0.165     | 0.370     | -0.445 | 0.658  | No significant difference between seasonal and non-seasonal counties |
| income_growth_pct             | -0.616     | 0.013     | -45.989| 0.000  | Higher income growth → more affordable housing |
| home_value_growth_pct         | 0.618      | 0.014     | 43.817 | 0.000  | Higher home price growth → less affordable housing |
| housing_units_growth_pct      | 0.062      | 0.035     | 1.806  | 0.076  | Not statistically significant at 0.05 level |
| affordability_index_2013      | -0.342     | 0.318     | -1.078 | 0.285  | Starting affordability (2013) not significant |

**Model Fit:**  
- R-squared = 0.990, Adj. R-squared = 0.989  
- F-statistic = 1086, p < 0.001  

**Interpretation:**  
This analysis shows that **changes in income and home values are the main drivers of affordability**, with higher income growth improving affordability and higher home price growth reducing it. Measures of seasonal housing do **not** significantly influence changes in affordability.  

**Next Steps:**  
The focus will shift to **predictive modeling** to forecast future affordability trends. This will help identify counties at risk of becoming less affordable, without delving into the underlying causes of income or home price changes.


##Machine Learning

In [811]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#Shortening the name for fun
df = growth_df

#Creating features and targets for 2013->2018
X_2013_2018 = df[[
    'median_household_income_2013',
    'median_home_value_2013',
    'pct_seasonal_2013',
    'total_seasonal_units_2013',
    'total_housing_units_2013',
    'seasonal_indicator'
]]
y_2013_2018 = ((df['affordability_index_2018'] / df['affordability_index_2013']) - 1) * 100

# ---- b) 2013+2018 -> 2023 ----
X_2013_2018_to_2023 = df[[
    'median_household_income_2013', 'median_home_value_2013',
    'median_household_income_2018', 'median_home_value_2018',
    'pct_seasonal_2013', 'pct_seasonal_2018',
    'income_growth_pct', 'home_value_growth_pct',
    'housing_units_growth_pct', 'seasonal_units_growth_pct',
    'seasonal_indicator'
]]
y_2018_2023 = ((df['affordability_index_2023'] / df['affordability_index_2018']) - 1) * 100

#training random forest models

rf_2013_2018 = RandomForestRegressor(
    n_estimators=500, max_depth=5, random_state=42
)
rf_2013_2018.fit(X_2013_2018, y_2013_2018)
y_pred_2018 = rf_2013_2018.predict(X_2013_2018)

rf_2013_2018_to_2023 = RandomForestRegressor(
    n_estimators=500, max_depth=5, random_state=42
)
rf_2013_2018_to_2023.fit(X_2013_2018_to_2023, y_2018_2023)
y_pred_2023 = rf_2013_2018_to_2023.predict(X_2013_2018_to_2023)

#evaluating performance
def evaluate(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    print(f"=== {name} ===")
    print(f"MAE: {mae:.3f}")
    print(f"RMSE: {rmse:.3f}")
    print(f"R²: {r2:.3f}\n")

evaluate(y_2013_2018, y_pred_2018, "Random Forest: 2013 -> 2018")
evaluate(y_2018_2023, y_pred_2023, "Random Forest: 2013+2018 -> 2023")

#Checking feature importance
def show_feature_importance(model, X):
    importance = pd.DataFrame({
        'Feature': X.columns,
        'Importance': model.feature_importances_
    }).sort_values('Importance', ascending=False)
    return importance

print("\n--- 2013 -> 2018 Feature Importance ---")
print(show_feature_importance(rf_2013_2018, X_2013_2018))

print("\n--- 2013+2018 -> 2023 Feature Importance ---")
print(show_feature_importance(rf_2013_2018_to_2023, X_2013_2018_to_2023))


=== Random Forest: 2013 -> 2018 ===
MAE: 1.870
RMSE: 2.250
R²: 0.693

=== Random Forest: 2013+2018 -> 2023 ===
MAE: 1.962
RMSE: 2.591
R²: 0.900


--- 2013 -> 2018 Feature Importance ---
                        Feature  Importance
3     total_seasonal_units_2013    0.258472
1        median_home_value_2013    0.222806
2             pct_seasonal_2013    0.194577
0  median_household_income_2013    0.185025
4      total_housing_units_2013    0.128960
5            seasonal_indicator    0.010161

--- 2013+2018 -> 2023 Feature Importance ---
                         Feature  Importance
6              income_growth_pct    0.328823
7          home_value_growth_pct    0.273443
2   median_household_income_2018    0.090443
8       housing_units_growth_pct    0.070085
4              pct_seasonal_2013    0.045181
0   median_household_income_2013    0.041149
5              pct_seasonal_2018    0.040160
1         median_home_value_2013    0.039385
9      seasonal_units_growth_pct    0.036245
3         

### Random Forest Model Results

I trained two Random Forest models to see how county-level housing affordability in Wisconsin has changed over time. The first model used **2013 data to predict growth through 2018**, and the second used **2013+2018 data to predict growth through 2023**.  

**How the models did:**  

| Model | MAE | RMSE | R² |
|-------|-----|------|----|
| 2013 → 2018 | 1.87 | 2.25 | 0.69 |
| 2013+2018 → 2023 | 1.96 | 2.59 | 0.90 |

So, the first model explains about 69% of the variation in affordability growth, while the second does even better, explaining 90%. Not bad! The errors are reasonable for the scale of these changes.  

**What mattered most:**  

**2013 → 2018**  
| Feature | Importance |
|---------|-----------|
| total_seasonal_units_2013 | 0.26 |
| median_home_value_2013 | 0.22 |
| pct_seasonal_2013 | 0.19 |
| median_household_income_2013 | 0.19 |
| total_housing_units_2013 | 0.13 |
| seasonal_indicator | 0.01 |

**2013+2018 → 2023**  
| Feature | Importance |
|---------|-----------|
| income_growth_pct | 0.33 |
| home_value_growth_pct | 0.27 |
| median_household_income_2018 | 0.09 |
| housing_units_growth_pct | 0.07 |
| pct_seasonal_2013 | 0.05 |
| median_household_income_2013 | 0.04 |
| pct_seasonal_2018 | 0.04 |
| median_home_value_2013 | 0.04 |
| seasonal_units_growth_pct | 0.04 |
| median_home_value_2018 | 0.03 |
| seasonal_indicator | 0.001 |

**What this tells us:**  
- For 2013 → 2018, seasonal housing (both total units and percent of housing) was a big driver of affordability changes. Counties with more seasonal homes saw bigger shifts.  
- For 2013+2018 → 2023, recent **income and home value growth** really take over. What’s happening right now matters more than the older seasonal patterns.  
- The seasonal indicator itself barely mattered, which makes sense—it’s the actual size of seasonal housing that counts, not just labeling a county as “seasonal.”  

**Next steps:**  
- We can use these models to **predict 2028 affordability for every county**.  
- Just a reminder: these models tell us *how affordability is likely to change*, not *why* income or home prices are changing. Figuring out the drivers would be a whole other project.


In [812]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Ensure columns are in the exact same order as training
# To predict 2023-2028 affordability growth using the model trained for 2018-2023 (rf_2013_2018_to_2023),
# we need to construct the input features (X_2023_to_2028) by shifting the temporal context.
# The model was trained with features from 2013, 2018, and 2013-2023 growth rates.
# For predicting the next period, we'll use 2018 values as the 'past' (like 2013 was before),
# 2023 values as the 'current' (like 2018 was before), and the overall 2013-2023 growth rates
# as a projection for the next 5-year period.

# Create a new DataFrame for X_2023_to_2028 with column names matching the training data
# but populated with the shifted time values.
X_2023_to_2028_corrected = pd.DataFrame({
    'median_household_income_2013': df['median_household_income_2018'],
    'median_home_value_2013': df['median_home_value_2018'],
    'median_household_income_2018': df['median_household_income_2023'],
    'median_home_value_2018': df['median_home_value_2023'],
    'pct_seasonal_2013': df['pct_seasonal_2018'],
    'pct_seasonal_2018': df['pct_seasonal_2023'],
    'income_growth_pct': df['income_growth_pct'],
    'home_value_growth_pct': df['home_value_growth_pct'],
    'housing_units_growth_pct': df['housing_units_growth_pct'],
    'seasonal_units_growth_pct': df['seasonal_units_growth_pct'],
    'seasonal_indicator': df['seasonal_indicator']
})

# Ensure the column order is the same as the training data used for rf_2013_2018_to_2023
X_2023_to_2028 = X_2023_to_2028_corrected[X_2013_2018_to_2023.columns]

# Now predict
df['affordability_index_growth_2023_2028'] = rf_2013_2018_to_2023.predict(X_2023_to_2028)

# Compute projected 2028 affordability index
df['affordability_index_2028'] = df['affordability_index_2023'] * (
    1 + df['affordability_index_growth_2023_2028'] / 100
)

# Prepare Tableau-ready long format
df_long = pd.melt(
    df,
    id_vars=['County'],
    value_vars=[
        'affordability_index_2013',
        'affordability_index_2018',
        'affordability_index_2023',
        'affordability_index_2028'
    ],
    var_name='Year',
    value_name='Affordability_Index'
)

# Clean up Year column
df_long['Year'] = df_long['Year'].str.extract(r'(\d{4})').astype(int)

df_long.tail()

final_affordability_predictions = df_long

In [813]:
final_affordability_predictions.tail()

,County,Year,Affordability_Index
283,Dane,2028,4.904381
284,Calumet,2028,3.365677
285,Ozaukee,2028,4.365144
286,St. Croix,2028,4.055547
287,La Crosse,2028,3.977556


##Final Results:

With these predictions I can see where affordability is headed in the state. Although my hypothesis appears to be incorrect, as many seasonal-heavy counties are seeing affordability increase this is only one piece of the puzzle. There are many ways in which this project could take the step, and maybe one day I will take that step. Questions such as:
- What is causing income/home value growth or decay?
  - Demographic factors
  - Economic factors
- How would gathering more years of data improve the results?
- What other key variables are missing?
- Did COVID have an impact on this? Are remote workers staying in their formally seasonal homes?

Like I said, many questions? For now, I encourage you to take a look at the interactive Tableau dashboard that I have published on my Github and allows you to take an interactive look through the data.

###Getting the data ready for Tableau

In [814]:
historical_years = [2013, 2018, 2023]
historical_df = final_df[final_df['year'].isin(historical_years)].copy()
historical_df['Affordability_Index'] = historical_df['median_home_value'] / historical_df['median_household_income']
historical_df.sort_values(by=['FIPS'], ascending=[True])

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal,FIPS,Affordability_Index
0,Adams,20741,2013,44897,17358,7465,135600,43.01,1,3.020246
72,Adams,20073,2018,43280,17672,7781,125100,44.03,1,2.890481
144,Adams,20928,2023,59153,16873,6473,184500,38.36,1,3.119030
73,Ashland,15712,2018,43861,9657,2215,110100,22.94,3,2.510203
145,Ashland,16050,2023,57645,9402,1755,160500,18.67,3,2.784283
...,...,...,...,...,...,...,...,...,...,...
70,Winnebago,167860,2013,51010,73496,1288,141300,1.75,139,2.770045
142,Winnebago,169926,2018,57124,75203,1234,149500,1.64,139,2.617114
71,Wood,74469,2013,47685,34117,395,119300,1.16,141,2.501835
143,Wood,73274,2018,53473,34868,522,129900,1.50,141,2.429263


In [815]:
import pandas as pd


pred_2028 = final_affordability_predictions.copy()


pred_2028 = pred_2028.rename(columns={'Year': 'year'})

pred_2028 = pred_2028[pred_2028['year'] == 2028]


pred_2028 = pred_2028.drop_duplicates(subset=['County'])


fips_pop_2023 = (
    final_df[final_df['year'] == 2023]
    [['County', 'FIPS', 'total_population']]
    .drop_duplicates(subset=['County'])
)

pred_2028 = pred_2028.merge(
    fips_pop_2023,
    on='County',
    how='left'
)

#Add empty historical columns
for col in [
    'median_household_income',
    'total_housing_units',
    'total_seasonal_units',
    'median_home_value',
    'pct_seasonal'
]:
    pred_2028[col] = pd.NA
#Final column order
pred_2028 = pred_2028[
    [
        'FIPS',
        'County',
        'year',
        'total_population',
        'median_household_income',
        'total_housing_units',
        'total_seasonal_units',
        'median_home_value',
        'pct_seasonal',
        'Affordability_Index'
    ]
]

pred_2028


,FIPS,County,year,total_population,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal,Affordability_Index
0,78,Menominee,2028,4256,<NA>,<NA>,<NA>,<NA>,<NA>,1.390015
1,125,Vilas,2028,23410,<NA>,<NA>,<NA>,<NA>,<NA>,3.821021
2,137,Waushara,2028,24752,<NA>,<NA>,<NA>,<NA>,<NA>,3.015964
3,85,Oneida,2028,38007,<NA>,<NA>,<NA>,<NA>,<NA>,3.446848
4,31,Douglas,2028,44197,<NA>,<NA>,<NA>,<NA>,<NA>,2.687308
...,...,...,...,...,...,...,...,...,...,...
67,25,Dane,2028,564777,<NA>,<NA>,<NA>,<NA>,<NA>,4.904381
68,15,Calumet,2028,52626,<NA>,<NA>,<NA>,<NA>,<NA>,3.365677
69,89,Ozaukee,2028,92345,<NA>,<NA>,<NA>,<NA>,<NA>,4.365144
70,109,St. Croix,2028,94819,<NA>,<NA>,<NA>,<NA>,<NA>,4.055547


In [816]:
#Now merging the two dataframes
final_long_df = pd.concat([historical_df, pred_2028], axis=0)
final_long_df

/tmp/ipython-input-2913514436.py:2: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_long_df = pd.concat([historical_df, pred_2028], axis=0)


,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal,FIPS,Affordability_Index
0,Adams,20741,2013,44897,17358,7465,135600,43.01,1,3.020246
1,Ashland,16068,2013,38550,9625,2313,107400,24.03,3,2.785992
2,Barron,45826,2013,44054,23598,2952,134900,12.51,5,3.062151
3,Bayfield,15071,2013,44944,13037,5273,158600,40.45,7,3.528836
4,Brown,250597,2013,53119,105151,531,158700,0.50,9,2.987632
...,...,...,...,...,...,...,...,...,...,...
67,Dane,564777,2028,<NA>,<NA>,<NA>,<NA>,NaN,25,4.904381
68,Calumet,52626,2028,<NA>,<NA>,<NA>,<NA>,NaN,15,3.365677
69,Ozaukee,92345,2028,<NA>,<NA>,<NA>,<NA>,NaN,89,4.365144
70,St. Croix,94819,2028,<NA>,<NA>,<NA>,<NA>,NaN,109,4.055547


In [817]:
# For final_df and pred_2028
state_code = '55'  # Wisconsin
final_long_df['FIPS'] = state_code + final_long_df['FIPS'].astype(str).str.zfill(3)


In [818]:
final_long_df

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal,FIPS,Affordability_Index
0,Adams,20741,2013,44897,17358,7465,135600,43.01,55001,3.020246
1,Ashland,16068,2013,38550,9625,2313,107400,24.03,55003,2.785992
2,Barron,45826,2013,44054,23598,2952,134900,12.51,55005,3.062151
3,Bayfield,15071,2013,44944,13037,5273,158600,40.45,55007,3.528836
4,Brown,250597,2013,53119,105151,531,158700,0.50,55009,2.987632
...,...,...,...,...,...,...,...,...,...,...
67,Dane,564777,2028,<NA>,<NA>,<NA>,<NA>,NaN,55025,4.904381
68,Calumet,52626,2028,<NA>,<NA>,<NA>,<NA>,NaN,55015,3.365677
69,Ozaukee,92345,2028,<NA>,<NA>,<NA>,<NA>,NaN,55089,4.365144
70,St. Croix,94819,2028,<NA>,<NA>,<NA>,<NA>,NaN,55109,4.055547


In [819]:
from google.colab import files

# Save CSV to Colab root temporarily
final_long_df.to_csv('wisconsin_affordability.csv', index=False)

# Download to your computer
files.download('wisconsin_affordability.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [820]:
final_long_df.sort_values(by=['FIPS', 'year'])

,County,total_population,year,median_household_income,total_housing_units,total_seasonal_units,median_home_value,pct_seasonal,FIPS,Affordability_Index
0,Adams,20741,2013,44897,17358,7465,135600,43.01,55001,3.020246
72,Adams,20073,2018,43280,17672,7781,125100,44.03,55001,2.890481
144,Adams,20928,2023,59153,16873,6473,184500,38.36,55001,3.119030
45,Adams,20928,2028,<NA>,<NA>,<NA>,<NA>,NaN,55001,3.312829
1,Ashland,16068,2013,38550,9625,2313,107400,24.03,55003,2.785992
...,...,...,...,...,...,...,...,...,...,...
47,Winnebago,171357,2028,<NA>,<NA>,<NA>,<NA>,NaN,55139,3.176888
71,Wood,74469,2013,47685,34117,395,119300,1.16,55141,2.501835
143,Wood,73274,2018,53473,34868,522,129900,1.50,55141,2.429263
215,Wood,74039,2023,66417,34662,627,168100,1.81,55141,2.530979
